In [1]:
import json
import logging
from sierra_ils_utils import SierraAPI, RecordDateRange, endpoints

# Configure the root logger
logging.basicConfig(level=logging.DEBUG)

# Load configuration
with open('./.config.json') as f:
    config = json.load(f)

# Create and configure the logger for the current module
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Optionally, set the logging level for the sierra_ils_utils module
logging.getLogger('sierra_ils_utils').setLevel(logging.DEBUG)

# Initialize SierraRESTAPI with DEBUG logging level
sierra_api = SierraAPI(
    sierra_api_base_url=config.get('sierra_api_base_url'),
    sierra_api_key=config.get('sierra_api_key'),
    sierra_api_secret=config.get('sierra_api_secret'),
    log_level=logging.DEBUG
)

In [2]:
endpoints['POST']

{'bibs/query': {'\n            Filter the records by a query in JSON format\n            responses': {200: sierra_ils_utils.sierra_api_v6_endpoints.QueryResultSet,
   400: sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode,
   404: sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode},
  'response_model': sierra_ils_utils.sierra_api_v6_endpoints.QueryResultSet,
  'body_required': True,
  'body_format': 'json'}}

In [3]:
response = sierra_api.post(
    'bibs/query',
    params={
        'offset': 0,
        'limit': 1,
    },
    json_body='{}'
)

print(response.data)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): classic.cincinnatilibrary.org:443
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "POST /iii/sierra-api/v6/token HTTP/1.1" 200 230
INFO:sierra_ils_utils.sierra_ils_utils:Authorization Success. response.json.get('expires_in'): 3600
INFO:sierra_ils_utils.sierra_ils_utils:Sierra session authenticated
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/info/token HTTP/1.1" 200 3584
INFO:sierra_ils_utils.sierra_ils_utils:Sierra response status code                  : 200
INFO:sierra_ils_utils.sierra_ils_utils:Sierra 'expiresIn'                           : 3599
INFO:sierra_ils_utils.sierra_ils_utils:session expires at (UNIX Epoch)              : 1701280204.3908648
INFO:sierra_ils_utils.sierra_ils_utils:seconds left                                 : 3539.982747554779
INFO:sierra_ils_utils.sierra_ils_utils:request url                                  : https://class

total=1 start=0 entries=[QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1000001')]


In [4]:
response = sierra_api.post(
    'bibs/query',
    params={
        'offset': 0,
        'limit': 1,
    },
    json_body={}
)

print(response.data)

DEBUG:sierra_ils_utils.sierra_ils_utils:"template": bibs/query
DEBUG:sierra_ils_utils.sierra_ils_utils:POST {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/query", "params": "{'offset': 0, 'limit': 1}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "POST /iii/sierra-api/v6/bibs/query?offset=0&limit=1 HTTP/1.1" 200 113
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 2
DEBUG:sierra_ils_utils.sierra_ils_utils:POST https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/query?offset=0&limit=1 200 ✅


total=1 start=0 entries=[QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1000001')]


In [7]:
# the create lists json query ...
json_body = """\
{
  "target": {
    "record": {"type": "bib"},
    "field": {"tag": "t"}
  },
  "expr": {
    "op": "has",
    "operands": ["spider-man"]
  }
}
"""

response = sierra_api.post(
    'bibs/query',
    params={
        'offset': 0,
        'limit': 10,
    },
    json_body=json_body
)

print(response.data)

DEBUG:sierra_ils_utils.sierra_ils_utils:"template": bibs/query
DEBUG:sierra_ils_utils.sierra_ils_utils:POST {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/query", "params": "{'offset': 0, 'limit': 20}"}
DEBUG:urllib3.connectionpool:Resetting dropped connection: classic.cincinnatilibrary.org
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "POST /iii/sierra-api/v6/bibs/query?offset=0&limit=20 HTTP/1.1" 200 1634
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 5
DEBUG:sierra_ils_utils.sierra_ils_utils:POST https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/query?offset=0&limit=20 200 ✅


total=20 start=0 entries=[QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1061673'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1069164'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1382468'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1527184'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1643908'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1789847'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1806283'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1859336'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1871452'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1886313'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/190686

In [9]:
if response.data.total and response.data.total > 0:
    for i, entry in enumerate(response.data.entries, start=1):
        print(i, entry.link)
        bib_response = sierra_api.get(
            'bibs/{id}',
            params={
                'fields': 'title'
            },
            path_params={
                'id': entry.link.split('/')[-1]  # just need the id or record num portion
            }
        )

        print(bib_response.data.title)





# sierra_api.get(
#     'bibs/{id}',
#     params={
#         'fields': 'default'
#     },
#     path_params= {
#         'id': 1081677
#     }
# )

DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1061673", "params": "{'fields': 'title'}"}
DEBUG:urllib3.connectionpool:Resetting dropped connection: classic.cincinnatilibrary.org
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1061673?fields=title HTTP/1.1" 200 84
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 6
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1061673?fields=title 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1069164", "params": "{'fields': 'title'}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1069164?fields=title HTTP/1.1" 200 54
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 7
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cinci

1 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1061673
Superman and Spider-Man : the superheroes on screen files
2 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1069164
Himmler ... Nazi spider-man
3 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1382468


DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1382468?fields=title HTTP/1.1" 200 77
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 8
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1382468?fields=title 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1527184", "params": "{'fields': 'title'}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1527184?fields=title HTTP/1.1" 200 50
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 9
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1527184?fields=title 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1643908", "params": "{'fields': 'title'}"}
DEBUG:urllib3.connecti

Marvel masterworks presents the Amazing Spider-Man
4 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1527184
The ultimate Spider-Man
5 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1643908
Spider-Man : the Venom factor
6 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1789847
The story of Spider-Man
7 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1806283
Spider-Man [sound recording] : music from and inspired by.
8 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1859336


DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1859336?fields=title HTTP/1.1" 200 86
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 13
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1859336?fields=title 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1871452", "params": "{'fields': 'title'}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1871452?fields=title HTTP/1.1" 200 66
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 14
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1871452?fields=title 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1886313", "params": "{'fields': 'title'}"}
DEBUG:urllib3.connec

Spider-Man confidential : from comic icon to Hollywood hero
9 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1871452
Spider-Man : secret of the Sinister Six
10 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1886313
I am Spider-Man
11 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1906862
The amazing Spider-Man.
12 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1913027
The very best of Spider-Man
13 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1915377


DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1915377?fields=title HTTP/1.1" 200 49
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 18
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1915377?fields=title 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1915416", "params": "{'fields': 'title'}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1915416?fields=title HTTP/1.1" 200 47
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 19
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1915416?fields=title 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1990292", "params": "{'fields': 'title'}"}
DEBUG:urllib3.connec

The amazing Spider-Man
14 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1915416
Ultimate Spider-Man.
15 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1990292
Spider-Man [videorecording]
16 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/2017640
The Spider-Man file : comics file magazine spotlight on the Spider-Man file
17 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/2213287
The spectacular Spider-Man
18 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/2258986
Spider-Man : blue
19 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/2282092


DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/2282092?fields=title HTTP/1.1" 200 84
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 24
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/2282092?fields=title 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/2368950", "params": "{'fields': 'title'}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/2368950?fields=title HTTP/1.1" 200 59
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 25
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/2368950?fields=title 200 ✅


Caught in the web : dreaming up the world of Spider-Man 2
20 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/2368950
Spider-Man ultimate picture book


In [12]:
type(json_data)

dict

In [2]:
# the json body data
json_data_test1 = "{}"
json_data_test2 = """\
{
  "target": {
    "record": {"type": "bib"},
    "field": {"tag": "t"}
  },
  "expr": {
    "op": "equals",
    "operands": ["moby dick"]
  }
}
"""

sierra_api.put(
    'bibs/query',
    params={
        'offset': 0,
        'limit': 1,
        'json': json_data_test1
    }
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): classic.cincinnatilibrary.org:443
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "POST /iii/sierra-api/v6/token HTTP/1.1" 200 230
INFO:sierra_ils_utils.sierra_ils_utils:Authorization Success. response.json.get('expires_in'): 3600
INFO:sierra_ils_utils.sierra_ils_utils:Sierra session authenticated
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/info/token HTTP/1.1" 200 3584
INFO:sierra_ils_utils.sierra_ils_utils:Sierra response status code                  : 200
INFO:sierra_ils_utils.sierra_ils_utils:Sierra 'expiresIn'                           : 3599
INFO:sierra_ils_utils.sierra_ils_utils:session expires at (UNIX Epoch)              : 1701203011.189504
INFO:sierra_ils_utils.sierra_ils_utils:seconds left                                 : 3539.981072664261
INFO:sierra_ils_utils.sierra_ils_utils:request url                                  : https://classi

Exception: GET response non-200 : {"code":106,"specificCode":0,"httpStatus":400,"name":"Parameter not recognized"}

In [5]:
# get some items data
sierra_api.get(
    'bibs/query',   # endpoint
    params= {       # query params
        # 'fields': 'default',
        'offset': 1,
        'limit': 1,
        'json': '{}'
    }
)

DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/query", "body": "{}" ,"params": "{}"}
DEBUG:urllib3.connectionpool:Resetting dropped connection: classic.cincinnatilibrary.org
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/query?offset=1&limit=1 HTTP/1.1" 400 80
ERROR:sierra_ils_utils.sierra_ils_utils:Error: {"code":106,"specificCode":0,"httpStatus":400,"name":"Parameter not recognized"}


Exception: GET response non-200 : {"code":106,"specificCode":0,"httpStatus":400,"name":"Parameter not recognized"}